In [1]:
import streamlit as st
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Step 1: Define the Deep Triangle Model
def build_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Dense(64, activation="relu"),
        Dropout(0.2),
        Dense(32, activation="relu"),
        Dense(16, activation="relu"),
        Dense(1)
    ])
    model.compile(optimizer="adam", loss="mse", metrics=["mae"])
    return model

# Step 2: Preprocess the Data
def preprocess_data(df):
    # Melt the triangle into long format
    df_long = df.melt(id_vars="Accident Year", 
                      var_name="Development Period", 
                      value_name="Claims")
    
    # Convert columns to appropriate types
    df_long["Accident Year"] = df_long["Accident Year"].astype(int)
    df_long["Development Period"] = df_long["Development Period"].astype(int)
    df_long["Claims"] = pd.to_numeric(df_long["Claims"], errors="coerce")
    
    # Drop rows with missing claims
    df_long = df_long.dropna(subset=["Claims"])
    
    # Normalize the Claims column
    scaler = StandardScaler()
    df_long["Normalized Claims"] = scaler.fit_transform(df_long[["Claims"]])
    
    return df_long, scaler

# Step 3: Train or Load the Model
@st.cache_resource
def train_or_load_model(data):
    X = data[["Accident Year", "Development Period"]]
    y = data["Normalized Claims"]
    
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Build and train the model
    model = build_model(input_shape=(2,))
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32, verbose=0)
    
    return model, X_test, y_test

# Step 4: Predict Using the Model
def predict(model, scaler, input_data):
    # Preprocess the input data
    input_data_long, _ = preprocess_data(input_data)
    X_input = input_data_long[["Accident Year", "Development Period"]]
    
    # Predict normalized claims
    y_pred_normalized = model.predict(X_input).flatten()
    
    # Reverse normalization
    y_pred = scaler.inverse_transform(y_pred_normalized.reshape(-1, 1)).flatten()
    
    # Add predictions to the input data
    input_data_long["Predicted Claims"] = y_pred
    return input_data_long

# Step 5: Streamlit App
def main():
    st.title("Deep Triangle Model for Loss Reserving")
    
    # File Upload
    uploaded_file = st.file_uploader("Upload a CSV file (loss triangle format)", type=["csv"])
    if uploaded_file is not None:
        # Load the uploaded data
        data = pd.read_csv(uploaded_file)
        
        # Preprocess the data
        processed_data, scaler = preprocess_data(data)
        
        # Train or load the model
        model, X_test, y_test = train_or_load_model(processed_data)
        
        # Display model performance
        y_pred_test = model.predict(X_test).flatten()
        mse = np.mean((y_pred_test - y_test) ** 2)
        st.write(f"Model MSE on Test Data: {mse:.2f}")
        
        # Make predictions
        predictions = predict(model, scaler, data)
        
        # Display predictions
        st.subheader("Predicted Claims")
        st.dataframe(predictions)
        
        # Visualize predictions
        st.subheader("Actual vs Predicted Claims")
        st.scatter_chart(
            predictions[["Claims", "Predicted Claims"]],
            x="Claims",
            y="Predicted Claims"
        )

if __name__ == "__main__":
    main()

2025-02-22 04:41:30.537 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 04:41:35.580 
  command:

    streamlit run C:\Users\Hp\miniforge3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-02-22 04:41:35.585 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 04:41:35.590 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 04:41:35.594 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 04:41:35.600 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 04:41:35.605 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 04:41:35.611 Thread 'MainThread': miss